In [295]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV
import shap
import rfpimp
from joblib import Parallel, delayed
from sklearn.model_selection import ParameterGrid
import datetime
import pandas as pd
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import check_password_hash, generate_password_hash
from sqlalchemy import Sequence
from sqlalchemy import Column, Integer, String, DateTime, Boolean
from flask_login import (current_user, LoginManager, login_required,
                         login_user, logout_user, UserMixin)

import matplotlib.pyplot as plt
from sqlalchemy import create_engine

## Feature engineering and data preparation

In [107]:
path = 'data/deep_odds/'

data_final = pd.read_pickle(path + 'data_final_form')

bookkeeper_data = pd.read_feather(path + 'bookkeeper-data')

# repetetive column
data_final.drop('league_id_y', axis=1, inplace=True)

data_final.drop(
    [
        'id', 'country_id', 'home_player_1', 'home_player_2', 'home_player_3',
        'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
        'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
        'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
        'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
        'away_player_9', 'away_player_10', 'away_player_11'
    ],
    axis=1,
    inplace=True)

data_final.drop(
    [
        'home_player_1_pos', 'home_player_2_pos', 'home_player_3_pos',
        'home_player_4_pos', 'home_player_5_pos', 'home_player_6_pos',
        'home_player_7_pos', 'home_player_8_pos', 'home_player_9_pos',
        'home_player_10_pos', 'home_player_11_pos', 'away_player_1_pos',
        'away_player_2_pos', 'away_player_3_pos', 'away_player_4_pos',
        'away_player_5_pos', 'away_player_6_pos', 'away_player_7_pos',
        'away_player_8_pos', 'away_player_9_pos', 'away_player_10_pos',
        'away_player_11_pos'
    ],
    axis=1,
    inplace=True)

data_final.drop(
    [
        'goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner',
        'possession'
    ],
    axis=1,
    inplace=True)

bk_filter = bookkeeper_data.iloc[:, :9]

data_final = pd.concat((data_final, bk_filter), axis=1)

data_final.dropna(how='any', inplace=True)

# * Training : upto last-2 seasons of each league
# * Validation : 2nd last season
# * Test: last season

# adding year column
data_final['year'] = data_final.date.dt.year

league_years = data_final[['league_id_x',
                           'year']].drop_duplicates().reset_index(drop=True)

# league_years

# validation set
val_yr_lg = league_years.groupby('league_id_x').tail(2).groupby(
    'league_id_x').head(1)
# val_yr_lg

# test set
test_yr_lg = league_years.groupby('league_id_x').tail(1)
# test_yr_lg

In [117]:
data_final['month'] = data_final.date.dt.month
data_final['dow'] = data_final.date.dt.dayofweek

In [119]:
# convert string to cats

convert_cats(data_final)

In [120]:
# removing nulls from int/float based columns, replace with -1 as this is what we did for string type also

data_final.fillna(-1, inplace=True)

In [121]:
# don't need id and date column

data_final.drop(['date'], axis=1, inplace=True)

In [122]:
data_final.drop(['home_team_goal', 'away_team_goal'], axis=1, inplace=True)

In [124]:
df_id = data_final.loc[:,['match_api_id', 'home_team_api_id',
       'away_team_api_id']]

In [127]:
id_cols_to_rem = list(data_final.filter(regex='id$').columns) + list(data_final.filter(regex='\d$').columns)

In [128]:
data_final.drop(id_cols_to_rem, axis=1, inplace=True)

In [129]:
# new cols were string
convert_cats(data_final)

In [130]:
df_bk = data_final.loc[:,['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA']]

In [131]:
data_final.drop(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA'],inplace=True,axis=1)

## Splitting data into train, validation and test 

In [132]:
train = data_final[(data_final.year != 2016) & (data_final.year != 2015) &
                   (data_final.year != 2014)]
val1 = data_final[data_final.year == 2014]
train_val1 = data_final[(data_final.year != 2016) & (data_final.year != 2015)]
val2 = data_final[data_final.year == 2015]

# completely hold out
test = data_final[data_final.year == 2016]

In [136]:
tst_idx = test.index

In [137]:
x_train, y_train = train.drop('outcome', axis=1), train['outcome']
x_val1, y_val1 = val1.drop('outcome', axis=1), val1['outcome']
x_val2, y_val2 = val2.drop('outcome', axis=1), val2['outcome']

## Random Forest Model

Multiclass without callibration

In [145]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=10)
m.fit(train_val1.drop('outcome', axis=1), train_val1['outcome'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

In [148]:
m.score(val2.drop('outcome', axis=1), val2['outcome'])

0.5110047846889952

Multiclass with callibration 

In [149]:
m2 = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=10)
m2.fit(train.drop('outcome', axis=1), train['outcome'])
m2_probs = m2.predict_proba(val1.drop('outcome', axis=1))

sig_m2 = CalibratedClassifierCV(m2, method="sigmoid", cv="prefit")
sig_m2.fit(val1.drop('outcome', axis=1),
           val1['outcome'])  # final callibrated model
sig_m2_probs = sig_m2.predict_proba(val2.drop('outcome', axis=1))
sig_score = log_loss(val2['outcome'], sig_m2_probs)

# print('logloss : ', sig_score)

logloss :  1.0004673910991777


In [150]:
sig_m2.score(x_val2, y_val2)

0.5052631578947369

**Hyperparameter tuning**

In [151]:
def fitOne(model, x_train, y_train, x_val1, y_val1, params):

    model.set_params(**params)
    m = model.fit(x_train, y_train)

    sig_mod = CalibratedClassifierCV(m, method="sigmoid", cv="prefit")
    return sig_mod.fit(x_val1, y_val1)


def fitModels(model,
              paramGrid,
              x_train,
              y_train,
              x_val1,
              y_val1,
              n_jobs=-1,
              verbose=10):

    return Parallel(
        n_jobs=n_jobs, verbose=verbose)(
            delayed(fitOne)(model, x_train, y_train, x_val1, y_val1, params)
            for params in paramGrid)


def scoreOne(model, x, y, metric):
    probs = model.predict_proba(x)
    return log_loss(y, probs)


def scoreModels(models,
                X,
                y,
                metric=log_loss,
                predict_proba=True,
                n_jobs=-1,
                verbose=10):

    return Parallel(
        n_jobs=n_jobs, verbose=np.ceil(verbose / 10))(
            delayed(scoreOne)(m, X, y, metric) for m in models)


def getBestModel(models, scores, greater_is_better=False):

    if greater_is_better:
        return models[np.argmax(scores)]
    else:
        return models[np.argmin(scores)]


def getBestScore(scores, greater_is_better=False):

    if greater_is_better:
        return np.max(scores)
    else:
        return np.min(scores)


def bestFit(model,
            paramGrid,
            x_train,
            y_train,
            x_val1,
            y_val1,
            x_val2,
            y_val2,
            metric=log_loss,
            greater_is_better=False,
            predict_proba=True,
            scoreLabel=None,
            vrange=None,
            cmap='YlOrRd',
            n_jobs=-1,
            verbose=10):
    print('-------------FITTING MODELS-------------')
    models = fitModels(model, paramGrid, x_train, y_train, x_val1, y_val1,
                       n_jobs, verbose)
    print('-------------SCORING MODELS-------------')
    scores = scoreModels(models, x_val2, y_val2, metric, predict_proba, n_jobs,
                         verbose)

    return getBestModel(models, scores, greater_is_better), getBestScore(
        scores, greater_is_better), models, scores

In [152]:
grid = {
    'min_samples_leaf': [1, 5, 10, 15, 20, 25],
    'max_features': ['sqrt', 'log2', 0.5],
    'n_estimators': [100],
    'n_jobs': [-1],
    'random_state': [10]
}
paramGrid = ParameterGrid(grid)

In [153]:
best_model, best_score, all_models, all_scores = bestFit(RandomForestClassifier(), paramGrid,
                                                    x_train, y_train, x_val1, y_val1, x_val2, y_val2,
                                                    metric=roc_auc_score, greater_is_better=False, 
                                                    scoreLabel='log_loss')

-------------FITTING MODELS-------------


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed:  1.4min remaining:   32.3s
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed:  1.9min remaining:   22.3s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  2.4min finished


-------------SCORING MODELS-------------


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    1.6s finished


In [156]:
# best_model.score(x_val2, y_val2)

0.5119617224880383

## Test prediction for postgre

In [251]:
def label_fix(x):
    if x[-1] == 'H': return 'home'
    elif x[-1] == 'A': return 'away'
    else: return 'draw'

In [157]:
x_test = test.drop('outcome', axis=1).values

In [158]:
test_pred = best_model.predict_proba(x_test)

In [159]:
test_pred = pd.DataFrame(test_pred, columns=['away', 'draw', 'home'])

In [163]:
test_id = df_id.loc[tst_idx].reset_index(drop=True)

In [169]:
test_bk = df_bk.loc[tst_idx].reset_index(drop=True)

In [182]:
test_pred.reset_index(drop=True, inplace=True)

In [191]:
df_final = pd.concat((test_id, test_bk, test_pred), axis=1, ignore_index=True)

In [195]:
df_final.columns = [
    'match_api_id', 'home_team_api_id', 'away_team_api_id', 'B365H', 'B365D',
    'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'away', 'draw', 'home'
]

In [203]:
df_final['away'] = 1 / df_final.away

In [205]:
df_final['home'] = 1 / df_final.home

In [206]:
df_final['draw'] = 1 / df_final.draw

In [250]:
tmp = pd.melt(
    df_final,
    id_vars=['match_api_id', 'home_team_api_id', 'away_team_api_id'],
    value_vars=['away', 'draw', 'home'],
    var_name='prediction',
    value_name='predicted_odds')

In [252]:
tmp1 = pd.melt(
    df_final,
    id_vars=['match_api_id', 'home_team_api_id', 'away_team_api_id'],
    value_vars=['B365H', 'B365D', 'B365A'],
    var_name='bookmaker',
    value_name='bookmaker_odds')

tmp1['prediction'] = tmp1.bookmaker.apply(label_fix)

tmp1['bookmaker'] = 'bet365'

In [253]:
tmp2 = pd.melt(
    df_final,
    id_vars=['match_api_id', 'home_team_api_id', 'away_team_api_id'],
    value_vars=['BWH', 'BWD', 'BWA'],
    var_name='bookmaker',
    value_name='bookmaker_odds')

tmp2['prediction'] = tmp2.bookmaker.apply(label_fix)

tmp2['bookmaker'] = 'betway'

In [254]:
tmp3 = pd.melt(
    df_final,
    id_vars=['match_api_id', 'home_team_api_id', 'away_team_api_id'],
    value_vars=['IWH', 'IWD', 'IWA'],
    var_name='bookmaker',
    value_name='bookmaker_odds')

tmp3['prediction'] = tmp3.bookmaker.apply(label_fix)

tmp3['bookmaker'] = 'interwetten'

In [255]:
tmp4 = pd.concat((tmp1, tmp2, tmp3), axis=0).reset_index(drop=True)

In [265]:
pred_final = pd.merge(
    tmp,
    tmp4,
    on=['match_api_id', 'home_team_api_id', 'away_team_api_id', 'prediction'],
    how='left',
    copy=False)

In [266]:
pred_final['prediction_type'] = 'result'

In [390]:
pred_final.head()

,match_api_id,home_team_api_id,away_team_api_id,prediction,predicted_odds,bookmaker,bookmaker_odds,prediction_type,pred_id
0,1988900,9825,10261,away,5.776156,bet365,11.0,result,default
1,1988900,9825,10261,away,5.776156,betway,11.0,result,default
2,1988900,9825,10261,away,5.776156,interwetten,9.0,result,default
3,1988901,9826,8455,away,1.719195,bet365,2.1,result,default
4,1988901,9826,8455,away,1.719195,betway,2.1,result,default


## Loading to postgre

In [286]:
db_uname = 'power_user'
db_pswd = 'deepodds_usf'
db_host = 'ec2-50-112-29-56.us-west-2.compute.amazonaws.com'
db_name = 'deepodds'

In [287]:
def create_connection_postgre():
    """
    Function to create SQL connection object
    :return: SQL Connection object
    """
    engine = create_engine('postgresql://{}:{}@{}/{}'.format(
        db_uname, db_pswd, db_host, db_name))
    conn = engine.connect()
    return conn

In [388]:
def update_active_status(conn):
    """
    Function to update is_active column to False
    :param SQL connection object
    :return: None
    """
    conn.execute(update_is_active)
    return None

In [393]:
def insert_preds(x):
    """
    Function to update predictions by inserting new rows
    :x: single row from the prediction dataframe
    :return: None
    """
    timestamp = str(pd.datetime.now())
    conn.execute(insert_into_portfolio.format(x[0], x[1], x[2],x[3],x[7],x[4],x[5],x[6],timestamp,timestamp))
    return None

In [394]:
def update_prediction(conn, predictions):
    update_active_status(conn)
    predictions.apply(insert_preds, axis=1)
    return None

In [353]:
conn = create_connection_postgre()

In [386]:
update_is_active = '''
UPDATE bet_suggestion SET is_active = true; 
'''

In [389]:
insert_predictions = '''
INSERT INTO bet_suggestion VALUES(default, {}, {}, {}, '{}', '{}', {}, '{}', {}, TRUE, NULL, NULL, NULL, NULL, NULL, '{}'::timestamp, '{}'::timestamp);
'''

In [ ]:
update_prediction(conn,pred_final)